# pandapower WLS State Estimation
This is an introduction into the usage of the pandapower state estimation module. It shows how to 
create measurements for a pandapower network and the perform the state estimation with the weighted least squares (WLS) algorithm.

## Example Network

We will be using the reference network from the book "Power System State Estimation" by Ali Abur and Antonio Gómez Expósito. 
It contains 3 buses with connecting lines between buses 1-2, 1-3 and 2-3. 8 measurements of different types enable WLS state estimation.

We first create this network in pandapower.

In [1]:
import pandapower as pp

net = pp.create_empty_network()

pp.create_bus(net, name="bus 1", vn_kv=1., index=1)
pp.create_bus(net, name="bus 2", vn_kv=1., index=2)
pp.create_bus(net, name="bus 3", vn_kv=1., index=3)

pp.create_ext_grid(net, 1)  # set the slack bus to bus 1

pp.create_line_from_parameters(net, 1, 3, 1, r_ohm_per_km=.01, x_ohm_per_km=.03, c_nf_per_km=0.,
                               imax_ka=1)
pp.create_line_from_parameters(net, 1, 3, 1, r_ohm_per_km=.02, x_ohm_per_km=.05, c_nf_per_km=0.,
                               imax_ka=1)
pp.create_line_from_parameters(net, 2, 3, 1, r_ohm_per_km=.03, x_ohm_per_km=.08, c_nf_per_km=0.,
                               imax_ka=1)

print("Created the pandapower network.")

Created the pandapower network.


## Adding Measurements

Measurements are defined via the pandapower *create_measurement* function. The function signature is as follows: *create_measurement(net, type, bus, value, std_dev, line=None)*. The argument *net* is the pandapower network, for which the new measurement should be added. The argument *type* sets the measurement type (value is given as a string). The argument *bus* (int) sets the bus, at which the measurement is placed. *Value* is a number describing the measurement value. *std_dev* sets the standard deviation for this measurement (in the same unit as the measurement value). If the measurement belongs to a line instead of a bus, the argument *line* is set to the number of the line, at which the measurement should be added. In this case, the argument *bus* should be set to the bus number, at which the measurement is located (either the *from_bus* or the *to_bus* of the line). If *from_bus* is set, the measurement is taken at the *from_bus* side of the line (*P_from_to*), if *to_bus* is set, the measurement is taken at the *to_bus* side of the line (*P_to_from*).


### Types of Measurements
Possible measurement types are: 
*vbus_pu* for voltage measurements (in per-unit)  
*pbus_kw* for active power measurements at a bus (in kW)  
*qbus_kvar* for reactive power measurements (in kVar)  
*pline_kw* for active power measurements at a line (in kW)  
*qline_kvar* for reactive power measurements at a line (in kVar)   
*iline_a* for electrical current measurements at a line (in A)   

Now we can add our measurements, which are valid for one point in time. We are adding two voltage magnitude measurements on buses 1 and 2. The standard deviations are 0.004 p.u.. We are also adding bus injection measurements for both P and Q on bus 2. Finally, we are adding line measurements for lines 0 and 1, both placed at the side of bus 1. The bus parameter defines the side at which the line measurement is positioned, the line argument is the index of the line.

In [2]:
from pandapower.estimation import state_estimation
from pandapower.create import create_measurement

create_measurement(net, "vbus_pu", 1, 1.006, .004)        # V at bus 1
create_measurement(net, "vbus_pu", 2, .968, .004)         # V at bus 2

create_measurement(net, "pbus_kw", 2, -501, 10)           # P at bus 2
create_measurement(net, "qbus_kvar", 2, -286, 10)         # Q at bus 2

create_measurement(net, "pline_kw", 1, 888, 8, line=0)    # Pline (bus 1 -> bus 2) at bus 1
create_measurement(net, "pline_kw", 1, 1173, 8, line=1)   # Pline (bus 1 -> bus 3) at bus 1
create_measurement(net, "qline_kvar", 1, 568, 8, line=0)  # Qline (bus 1 -> bus 2) at bus 1
create_measurement(net, "qline_kvar", 1, 663, 8, line=1)  # Qline (bus 1 -> bus 3) at bus 1

We can check if all measurements have been created correctly:

In [3]:
net.measurement

,type,bus,line,value,std_dev
0,vbus_pu,1.0,None,1.006,0.004
1,vbus_pu,2.0,None,0.968,0.004
2,pbus_kw,2.0,None,-501.000,10.000
3,qbus_kvar,2.0,None,-286.000,10.000
4,pline_kw,1.0,0,888.000,8.000
5,pline_kw,1.0,1,1173.000,8.000
6,qline_kvar,1.0,0,568.000,8.000
7,qline_kvar,1.0,1,663.000,8.000


## Performing the State Estimation

The measurements are now set. We have to initialize the starting voltage magnitude and voltage angles for the state estimator. In continous operation, this can be the result of the last state estimation. In our case, we set flat start conditions: 1.0 p.u. for voltage magnitude, 0.0 degree for voltage angles.

In [4]:
import numpy as np

v_start = np.array([1.0, 1.0, 1.0])
delta_start = np.array([0., 0., 0.])

And now run the state estimation. Afterwards, the result will be stored in the table res_bus_est.

In [5]:
wls = state_estimation()
wls.set_grid(net)
success = wls.estimate(v_start, delta_start)
print(success)

True


## Working with Results

We can show the voltage magnitude and angles directly:

In [6]:
net.res_bus_est.vm_pu

1    1.016126
2    0.958471
3    0.998070
Name: vm_pu, dtype: float64

In [7]:
net.res_bus_est.va_degree

1    0.000000
2   -2.745292
3   -0.859228
Name: va_degree, dtype: float64

The results match exactly with the results from the book. Nice!    
Let's look at the bus power injections, which are available in res_bus_est as well

In [8]:
net.res_bus_est.p_kw

1   -1030.374059
2     498.883558
3     511.424868
Name: p_kw, dtype: float64

In [9]:
net.res_bus_est.q_kvar

1   -615.130495
2    280.875819
3    279.536299
Name: q_kvar, dtype: float64

This concludes the small tutorial how to perform state estimation with a pandapower network.